In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
import pandas as pd

In [36]:
gdf_traffic = pd.read_csv('/content/drive/MyDrive/October 4/concate df/the_real_final_traffic.csv')
gdf_traffic.head()

,Datetime,WktGeom,Avg_Vol_Geo,Boro,street,geometry,x,y,cluster,holidays
0,2012-01-01,POINT (1003830.9 238599.1),41.246324,Bronx,RIVER AVENUE,POINT (-73.9292534248472 40.8215532819544),1003830.9,238599.1,2,False
1,2012-01-01,POINT (1004049 238390.9),39.777778,Bronx,GERARD AVENUE,POINT (-73.92846603546185 40.82098134580181),1004049.0,238390.9,2,False
2,2012-01-01,POINT (1004088.5 235075.3),95.380769,Bronx,EAST 138 STREET,POINT (-73.92833312241837 40.81188086767926),1004088.5,235075.3,2,False
3,2012-01-01,POINT (1004287.7 236711.1),51.384030,Bronx,EAST 144 STREET,POINT (-73.92760862621256 40.816370230717226),1004287.7,236711.1,2,False
4,2012-01-01,POINT (1004510.2 237462.4),127.368421,Bronx,EAST 149 STREET,POINT (-73.92680251970938 40.81843183006975),1004510.2,237462.4,2,False


In [37]:
projects_df = pd.read_csv('/content/drive/MyDrive/October 4/concate df/FinalProjectDB.csv')
projects_df.head()

,Open Date,Project Name,Seasonal/Always Open,Floor,Area (sq ft),Location,Source,Borough,Type_of_Project,Street,Latitude,Longitude,Geometry
0,21/09/2014,The High Line,Always Open,0,"7,656","Gansevoort St to 34th St, Manhattan",The High Line,Manhattan,Park,Gansevoort St to 34th St,40.7482,-74.0048,POINT (982920.014 211866.125)
1,03/11/2014,One World Trade Center,Always Open,94,"3,501,274","285 Fulton St, Manhattan",One World Trade Center,Manhattan,Office Building,285 Fulton St,40.7129,-74.0134,POINT (980535.156 199005.481)
2,01/05/2015,Whitney Museum of American Art,Always Open,8,"220,000","99 Gansevoort St, Manhattan",Whitney Museum,Manhattan,Cultural,99 Gansevoort St,40.7398,-74.0088,POINT (981811.385 208805.823)
3,01/07/2015,Spring Street Salt Shed,Always Open,2,"700,000","353 West St, Manhattan",NYC Department of Design and Construction,Manhattan,Infrastructure,353 West St,40.7262,-74.0106,POINT (981311.978 203850.972)
4,23/12/2015,432 Park Avenue,Always Open,85,"412,637","432 Park Ave, Manhattan",CIM Group,Manhattan,Residential,432 Park Ave,40.7618,-73.9718,POINT (992062.071 216822.268)


prepare

In [38]:
!pip install geopandas
!pip install shapely


In [39]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import shapely.wkt as wkt

In [40]:
# prompt: convert this data frame gdf_traffic1 into geopandas whit EPSG:32618

import pandas as pd
# Assuming gdf_traffic1 is your DataFrame
gdf_traffic['geometry'] = gdf_traffic['WktGeom'].apply(wkt.loads)

# Convert to GeoDataFrame with specified CRS
gdf_traffic = gpd.GeoDataFrame(gdf_traffic, geometry='geometry', crs='EPSG:32618')


In [41]:
# prompt: convert this data frame gdf_traffic1 into geopandas whit EPSG:32618

import pandas as pd
# Assuming gdf_traffic1 is your DataFrame
projects_df['geometry'] = projects_df['Geometry'].apply(wkt.loads)

# Convert to GeoDataFrame with specified CRS
projects_df = gpd.GeoDataFrame(projects_df, geometry='geometry', crs='EPSG:32618')


In [42]:
projects_df['Open Date'] = pd.to_datetime(projects_df['Open Date'], dayfirst=True)  # Ensure 'Open Date' is datetime type

### Buffer

In [43]:
# Step 2: Create a buffer of 1000 units around each project point
projects_df['geometry'] = projects_df['geometry'].buffer(3000)

In [44]:
# Step 3: Perform spatial join (left join with intersects predicate)
joined_df = gpd.sjoin(gdf_traffic, projects_df, how='left', predicate='intersects')

In [45]:
# Step 4: Add a new column 'project_active' based on whether the project is active at the time of the traffic data
joined_df['project_active'] = (joined_df['Open Date'] <= joined_df['Datetime'])
joined_df['project_active'] = joined_df['project_active'].fillna(False)  # Fill NaN values with False

In [46]:
# Step 5: Ensure the new DataFrame includes all columns from the project DataFrame
# The 'joined_df' now contains all columns from both the traffic and project DataFrames

In [47]:
joined_df

,Datetime,WktGeom,Avg_Vol_Geo,Boro,street,geometry,x,y,cluster,holidays,...,Area (sq ft),Location,Source,Borough,Type_of_Project,Street,Latitude,Longitude,Geometry,project_active
0,2012-01-01,POINT (1003830.9 238599.1),41.246324,Bronx,RIVER AVENUE,POINT (1003830.9 238599.1),1.003831e+06,238599.100000,2,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,2012-01-01,POINT (1004049 238390.9),39.777778,Bronx,GERARD AVENUE,POINT (1004049 238390.9),1.004049e+06,238390.900000,2,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,2012-01-01,POINT (1004088.5 235075.3),95.380769,Bronx,EAST 138 STREET,POINT (1004088.5 235075.3),1.004088e+06,235075.300000,2,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,2012-01-01,POINT (1004287.7 236711.1),51.384030,Bronx,EAST 144 STREET,POINT (1004287.7 236711.1),1.004288e+06,236711.100000,2,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,2012-01-01,POINT (1004510.2 237462.4),127.368421,Bronx,EAST 149 STREET,POINT (1004510.2 237462.4),1.004510e+06,237462.400000,2,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25228,2019-12-01,POINT (963516.0908896723 168867.51740561606),18.015504,Staten Island,FRONT STREET,POINT (963516.091 168867.517),9.635161e+05,168867.517406,7,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
25229,2019-12-01,POINT (963574.543398993 167638.25546891819),50.969072,Staten Island,CANAL STREET,POINT (963574.543 167638.255),9.635745e+05,167638.255469,7,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
25230,2019-12-01,POINT (963600.8025487016 168664.11409327856),28.273902,Staten Island,MURRAY HULBERT AVENUE,POINT (963600.803 168664.114),9.636008e+05,168664.114093,7,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
25231,2019-12-01,POINT (963648.9349859224 167768.45571785266),29.984496,Staten Island,FRONT STREET,POINT (963648.935 167768.456),9.636489e+05,167768.455718,7,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [48]:
joined_df.isna().sum()

,0
Datetime,0
WktGeom,0
Avg_Vol_Geo,0
Boro,0
street,0
geometry,0
x,0
y,0
cluster,0
holidays,0


In [49]:
# prompt: print when project_active == True

joined_df[joined_df['project_active'] == True]


,Datetime,WktGeom,Avg_Vol_Geo,Boro,street,geometry,x,y,cluster,holidays,...,Area (sq ft),Location,Source,Borough,Type_of_Project,Street,Latitude,Longitude,Geometry,project_active
3514,2019-01-01,POINT (1018707.4087402204 255350.78402253857),234.788826,Bronx,SOUTHERN BOULEVARD,POINT (1018707.409 255350.784),1.018707e+06,255350.784023,8,False,...,"10,890,000","2900 Southern Boulevard, Bronx",Cooper Robertson,Bronx,Educational,2900 Southern Boulevard,40.8617,-73.8741,POINT (1019074.971 253243.173),True
3515,2019-01-01,POINT (1018732.3651488698 255358.15606056896),231.743371,Bronx,SOUTHERN BOULEVARD,POINT (1018732.365 255358.156),1.018732e+06,255358.156061,8,False,...,"10,890,000","2900 Southern Boulevard, Bronx",Cooper Robertson,Bronx,Educational,2900 Southern Boulevard,40.8617,-73.8741,POINT (1019074.971 253243.173),True
3779,2019-11-01,POINT (1017204.2276374499 251579.89216445672),121.696759,Bronx,SOUTHERN BOULEVARD,POINT (1017204.228 251579.892),1.017204e+06,251579.892164,8,False,...,"10,890,000","2900 Southern Boulevard, Bronx",Cooper Robertson,Bronx,Educational,2900 Southern Boulevard,40.8617,-73.8741,POINT (1019074.971 253243.173),True
3780,2019-11-01,POINT (1017239.0043787222 251576.15452932936),65.892361,Bronx,SOUTHERN BOULEVARD,POINT (1017239.004 251576.155),1.017239e+06,251576.154529,8,False,...,"10,890,000","2900 Southern Boulevard, Bronx",Cooper Robertson,Bronx,Educational,2900 Southern Boulevard,40.8617,-73.8741,POINT (1019074.971 253243.173),True
3794,2019-11-01,POINT (1020736.0204754705 251614.27827495412),173.028935,Bronx,PELHAM PARKWAY,POINT (1020736.02 251614.278),1.020736e+06,251614.278275,8,False,...,"10,890,000","2900 Southern Boulevard, Bronx",Cooper Robertson,Bronx,Educational,2900 Southern Boulevard,40.8617,-73.8741,POINT (1019074.971 253243.173),True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23912,2019-11-01,POINT (997256.4782944638 211009.19693793653),42.930556,Queens,VERNON BOULEVARD,POINT (997256.478 211009.197),9.972565e+05,211009.196938,4,False,...,"193,045,560","47-40 Center Blvd, Queens",Queens Library,Queens,Cultural,47-40 Center Blvd,40.7455,-73.9581,POINT (995860.137 210885.169),True
23913,2019-11-01,POINT (997856.8927164853 210641.9230549897),84.870370,Queens,11 STREET,POINT (997856.893 210641.923),9.978569e+05,210641.923055,1,False,...,"96,500","Center Blvd & 51st Ave, Queens",NYC Parks,Queens,Park,Center Blvd & 51st Ave,40.7425,-73.9607,POINT (995140.189 209791.839),True
23913,2019-11-01,POINT (997856.8927164853 210641.9230549897),84.870370,Queens,11 STREET,POINT (997856.893 210641.923),9.978569e+05,210641.923055,1,False,...,"193,045,560","47-40 Center Blvd, Queens",Queens Library,Queens,Cultural,47-40 Center Blvd,40.7455,-73.9581,POINT (995860.137 210885.169),True
23914,2019-11-01,POINT (997888.2212560362 210630.7372643547),67.804398,Queens,11 STREET,POINT (997888.221 210630.737),9.978882e+05,210630.737264,1,False,...,"96,500","Center Blvd & 51st Ave, Queens",NYC Parks,Queens,Park,Center Blvd & 51st Ave,40.7425,-73.9607,POINT (995140.189 209791.839),True


In [62]:
# prompt: joined_df when street not  null

joined_df_street_not_null = joined_df[joined_df['street'].notnull()]
joined_df_street_not_null


,Datetime,WktGeom,Avg_Vol_Geo,Boro,street,geometry,x,y,cluster,holidays,...,Area (sq ft),Location,Source,Borough,Type_of_Project,Street,Latitude,Longitude,Geometry,project_active
0,2012-01-01,POINT (1003830.9 238599.1),41.246324,Bronx,RIVER AVENUE,POINT (1003830.9 238599.1),1.003831e+06,238599.100000,2,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,2012-01-01,POINT (1004049 238390.9),39.777778,Bronx,GERARD AVENUE,POINT (1004049 238390.9),1.004049e+06,238390.900000,2,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,2012-01-01,POINT (1004088.5 235075.3),95.380769,Bronx,EAST 138 STREET,POINT (1004088.5 235075.3),1.004088e+06,235075.300000,2,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,2012-01-01,POINT (1004287.7 236711.1),51.384030,Bronx,EAST 144 STREET,POINT (1004287.7 236711.1),1.004288e+06,236711.100000,2,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,2012-01-01,POINT (1004510.2 237462.4),127.368421,Bronx,EAST 149 STREET,POINT (1004510.2 237462.4),1.004510e+06,237462.400000,2,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25228,2019-12-01,POINT (963516.0908896723 168867.51740561606),18.015504,Staten Island,FRONT STREET,POINT (963516.091 168867.517),9.635161e+05,168867.517406,7,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
25229,2019-12-01,POINT (963574.543398993 167638.25546891819),50.969072,Staten Island,CANAL STREET,POINT (963574.543 167638.255),9.635745e+05,167638.255469,7,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
25230,2019-12-01,POINT (963600.8025487016 168664.11409327856),28.273902,Staten Island,MURRAY HULBERT AVENUE,POINT (963600.803 168664.114),9.636008e+05,168664.114093,7,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
25231,2019-12-01,POINT (963648.9349859224 167768.45571785266),29.984496,Staten Island,FRONT STREET,POINT (963648.935 167768.456),9.636489e+05,167768.455718,7,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


1. drop unnessery columns
2. deal with null values
3. save


In [58]:
joined_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 30623 entries, 0 to 25232
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Datetime              30623 non-null  object        
 1   WktGeom               30623 non-null  object        
 2   Avg_Vol_Geo           30623 non-null  float64       
 3   Boro                  30623 non-null  object        
 4   street                30623 non-null  object        
 5   geometry              30623 non-null  geometry      
 6   x                     30623 non-null  float64       
 7   y                     30623 non-null  float64       
 8   cluster               30623 non-null  int64         
 9   holidays              30623 non-null  bool          
 10  index_right           8719 non-null   float64       
 11  Open Date             8719 non-null   datetime64[ns]
 12  Project Name          8719 non-null   object        
 13  Seasonal/Alwa

In [83]:
columns_to_drop = ['Latitude', 'Longitude', 'Geometry', 'Source', 'Location', 'Project Name', 'Seasonal/Always Open', 'index_right', 'cluster', 'x', 'y', 'Street', 'WktGeom', 'geometry', 'Open Date', 'Borough']
new_df = joined_df.drop(columns=columns_to_drop, errors='ignore')

new_df

,Datetime,Avg_Vol_Geo,Boro,street,holidays,Floor,Area (sq ft),Type_of_Project,project_active
0,2012-01-01,41.246324,Bronx,RIVER AVENUE,False,NaN,NaN,NaN,False
1,2012-01-01,39.777778,Bronx,GERARD AVENUE,False,NaN,NaN,NaN,False
2,2012-01-01,95.380769,Bronx,EAST 138 STREET,False,NaN,NaN,NaN,False
3,2012-01-01,51.384030,Bronx,EAST 144 STREET,False,NaN,NaN,NaN,False
4,2012-01-01,127.368421,Bronx,EAST 149 STREET,False,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...
25228,2019-12-01,18.015504,Staten Island,FRONT STREET,False,NaN,NaN,NaN,False
25229,2019-12-01,50.969072,Staten Island,CANAL STREET,False,NaN,NaN,NaN,False
25230,2019-12-01,28.273902,Staten Island,MURRAY HULBERT AVENUE,False,NaN,NaN,NaN,False
25231,2019-12-01,29.984496,Staten Island,FRONT STREET,False,NaN,NaN,NaN,False


In [84]:
new_df['Type_of_Project'] = new_df['Type_of_Project'].fillna('No_Project')

In [86]:
new_df['Area (sq ft)'] = new_df['Area (sq ft)'].fillna("0")

In [87]:
new_df['Area (sq ft)'] = new_df['Area (sq ft)'].str.replace(',', '')

In [88]:
new_df['Area (sq ft)'] = pd.to_numeric(new_df['Area (sq ft)']).astype(int)

In [92]:
new_df['Floor'] = pd.to_numeric(new_df['Floor']).fillna(0).astype(int)

In [93]:
new_df.isna().sum()

,0
Datetime,0
Avg_Vol_Geo,0
Boro,0
street,0
holidays,0
Floor,0
Area (sq ft),0
Type_of_Project,0
project_active,0


In [94]:
# prompt: sample Type_of_Project not = No_Project

sampled_df = new_df[new_df['Type_of_Project'] != 'No_Project'].sample(n=10, random_state=42)  # Replace 10 with the desired sample size
sampled_df


,Datetime,Avg_Vol_Geo,Boro,street,holidays,Floor,Area (sq ft),Type_of_Project,project_active
4244,2012-10-01,247.399177,Manhattan,COLUMBUS AV,False,98,1285308,Residential,False
8361,2018-06-01,165.518750,Manhattan,65 ST TRANSVERSE,False,98,1285308,Residential,False
8986,2019-10-01,71.437500,Manhattan,LAFAYETTE STREET,False,80,2232984,Mixed-Use Development,True
4764,2013-08-01,135.929047,Manhattan,CENTRAL PARK S,True,77,750000,Mixed-Use Development,False
16611,2018-11-01,34.803571,Brooklyn,SOUTH 4 STREET,False,0,21528,Park,True
8438,2018-10-01,101.503472,Manhattan,STATE STREET,False,80,2232984,Mixed-Use Development,True
7048,2016-06-01,310.729167,Manhattan,7 AVENUE,False,2,255000,Train Station / Transit Hub,False
8069,2017-11-01,231.123563,Manhattan,CENTRAL PARK SOUTH,False,77,750000,Mixed-Use Development,False
5123,2013-12-01,86.954282,Manhattan,E 42 ST,False,77,1750212,Interactive Observation Deck,False
8307,2018-05-01,91.171875,Manhattan,1 AVENUE,False,77,1750212,Interactive Observation Deck,False


## Test

In [96]:
# prompt: save new_df

new_df.to_csv('/content/drive/MyDrive/October 4/concate df/4oct_final_data_before_encodeing.csv', index=False)


In [98]:
hwl = pd.read_csv('/content/drive/MyDrive/October 4/concate df/4oct_final_data_before_encodeing.csv')
hwl

,Datetime,Avg_Vol_Geo,Boro,street,holidays,Floor,Area (sq ft),Type_of_Project,project_active
0,2012-01-01,41.246324,Bronx,RIVER AVENUE,False,0,0,No_Project,False
1,2012-01-01,39.777778,Bronx,GERARD AVENUE,False,0,0,No_Project,False
2,2012-01-01,95.380769,Bronx,EAST 138 STREET,False,0,0,No_Project,False
3,2012-01-01,51.384030,Bronx,EAST 144 STREET,False,0,0,No_Project,False
4,2012-01-01,127.368421,Bronx,EAST 149 STREET,False,0,0,No_Project,False
...,...,...,...,...,...,...,...,...,...
30618,2019-12-01,18.015504,Staten Island,FRONT STREET,False,0,0,No_Project,False
30619,2019-12-01,50.969072,Staten Island,CANAL STREET,False,0,0,No_Project,False
30620,2019-12-01,28.273902,Staten Island,MURRAY HULBERT AVENUE,False,0,0,No_Project,False
30621,2019-12-01,29.984496,Staten Island,FRONT STREET,False,0,0,No_Project,False


In [99]:
hwl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30623 entries, 0 to 30622
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Datetime         30623 non-null  object 
 1   Avg_Vol_Geo      30623 non-null  float64
 2   Boro             30623 non-null  object 
 3   street           30623 non-null  object 
 4   holidays         30623 non-null  bool   
 5   Floor            30623 non-null  int64  
 6   Area (sq ft)     30623 non-null  int64  
 7   Type_of_Project  30623 non-null  object 
 8   project_active   30623 non-null  bool   
dtypes: bool(2), float64(1), int64(2), object(4)
memory usage: 1.7+ MB
